# Merging csv from search terms in papers with existing spreadsheet

Author: Anne-Flore Cabouat
Date: March 2023

In [ ]:
##required libraries

import pandas as pd
import numpy as np

##loading data

#csv file from spreadsheet !important! include index column
df = pd.read_csv("../results/round 3/round1+2.csv")
df_max_index = 1138 #biggest value in index for that file at the begining of the process

#csv file to merge
df_add = pd.read_csv("../results/round 3/paperFound-MERGED.csv")

#output csv file place and name
results_path = "../results/"
csv_out = "round1+2+3.csv"

In [ ]:
#preparing files
df_add["index"] = 0 #add index col to new df
df_Cols = df.columns.values.tolist()
df_add_Cols = df_add.columns.values.tolist()
print(f"Existing columns = {df_Cols}\n")
print(f"New file columns = {df_add_Cols}\n")

In [ ]:
df_Col_keep = [value for value in df_add_Cols if value in df_Cols] #used for both original df and new df_add
print(df_Col_keep)
merge_col = "doi link" #choose the column that will be the reference for merging
concatenate_cols = ["term"] #choose column(s) that will be concatenated (the others will be combined)

In [ ]:
#making sure that empty or None in merge_col take value from another specific col
def merge_col_empty_fill(df, merge_col, unique_col, string_clean):    
    for index, row in df.iterrows():
        doi_link = row['doi link']
        if pd.isnull(doi_link):
            doi_link = ''
        if doi_link == '':
            x_unique = f"Unknown DOI {df.loc[index, unique_col].replace(string_clean,'')}"
            df.loc[index, merge_col] = x_unique
            print(f"Replaced null for {x_unique}")
    return df

In [ ]:
unique_col = 'path'
string_clean = "../papers-xml-extractions/Vis-all_full_paper_pdfs-text-extraction-results/"
df = merge_col_empty_fill(df, merge_col, unique_col, string_clean)
df.head()

In [ ]:
unique_col = 'path'
string_clean = "../papers-xml-extractions/run 3 - vis 2022 full + vis 2021 conf/"
df_add = merge_col_empty_fill(df_add, merge_col, unique_col, string_clean)
df_add.head()

In [ ]:
#adding missing part to transform DOI into URL
doi_URL = 'http://dx.doi.org/'
link_clean_needed = False

def add_doi_url(row):
    
    #the list of checks to perform
    def check_doi_content(row):
        #change the nuber of checks if you add one
        nb_of_checks = 5
        
        global check_list
        check_list = [False]*nb_of_checks
        
        global i
        i = 0
        
        def to_True():
            global check_list
            global i
            check_list[i]=True
            i+=1
            
        
        #checks to make
        if isinstance(row, str):
            to_True()
            
            if row != '':
                to_True()
                            
            if 'Unknown' not in row:
                to_True()
                
            ##second to last check
            if row.count("http://dx.doi.org/")==0:
                to_True()
                
            #last check
            if row.count("http://dx.doi.org/")<=1:
                check_list[i]=False
            else:
                to_True()
        
        else:
            print("Err: not str")
            print(type(row))
        
        return check_list
       
    checks_done = check_doi_content(row)
    
    doi_URL_there = True #assuming there is a DOI link
    
    if checks_done[-2]:
        print("DOI was alone")
        doi_URL_there = False
    
    #removing http://dx.doi.org/ when multiple
    if not checks_done[-1]:
        global link_clean_needed
        link_clean_needed = True
        del checks_done[-1]
    
    #adding
    if all(checks_done) and not doi_URL_there:
        row = doi_URL+row
        print("DOI was alone, we added http://dx.doi.org/")
        
    return row

    
def link_clean(df, doi_col):
    df[doi_col] = df[doi_col].apply(lambda x: add_doi_url(x))
    global link_clean_needed
    if link_clean_needed:
        print(f'Some links have multiple "{doi_URL}" in URL for this df {df.info()} Please use the next line or manually replace "{doi_URL}{doi_URL}" with "{doi_URL}".')
    return df

In [ ]:
df = link_clean(df, merge_col)
df.head()

In [ ]:
df_add = link_clean(df_add, merge_col)
df_add.head()

In [ ]:
#if needed to replace some erroneous strings...
df['doi link']=df['doi link'].str.replace("http://dx.doi.org/http://dx.doi.org/","http://dx.doi.org/", regex=True)
df_add['doi link']=df_add['doi link'].str.replace("http://dx.doi.org/http://dx.doi.org/","http://dx.doi.org/", regex=True)

In [ ]:
#lists of cols to drop for each df
df_add_Col_drop = [x for x in df_add_Cols if x not in df_Col_keep]
df_Col_drop = [x for x in df_Cols if x not in df_Col_keep]

print(f"df delete {df_Col_drop}")
print(f"df_add delete {df_add_Col_drop}")

In [ ]:
def drop_Cols(df):
    df_Cols = df.columns.values.tolist()
    for n in df_Cols:
        if n not in df_Col_keep:
            print(df.columns.values.tolist())
            df.drop(n, inplace=True, axis=1)
            print(f"Removed {n} column")
    df = df[df_Col_keep]
    df.info() #arrange col order
    return df

In [ ]:
df = drop_Cols(df)
df_add = drop_Cols(df_add)

In [ ]:
#check
if df.columns.values.tolist() != df_add.columns.values.tolist():
    raise ValueError("The keep list and final col list don't match.")

In [ ]:
df.info()
df_add.info()

In [ ]:
def combine_columns(df):
    #columns to be combined
    combine_cols = [x for x in df_Col_keep if x not in concatenate_cols]
    combine_cols.remove(merge_col)
    print(combine_cols)
    #combine same columns
    for n in combine_cols :
        try:
            df[f'{n}_x'] = df[f'{n}_x'].combine_first(df[f'{n}_y']) #combining _x (left from merging) and _y (right from merging) columns into _x
            df.rename(columns = {f'{n}_x':f'{n}'}, inplace=True) #rename column _x
            del df[f'{n}_y'] #dropping the _y column
        except Exception as e:
            print(f"!! Couldn't combine columns {n}, error {e}")

    #building continued index for new elements

    global df_max_index

    df_max_index += 1
    df.info()
    df["index"] = df["index"].apply(lambda x: 0 if x is np.nan else x)

    def addIndex(row):
        global df_max_index
        if row == 0:
            row = df_max_index
            df_max_index += 1
        return row
    
    df["index"] = df["index"].apply(lambda x: addIndex(x))
    
    #concatenating
    for m in concatenate_cols:
        ## replacing NaN with empty strings first
        df[f'{m}_x'] = df[f'{m}_x'].apply(lambda x: '' if x is np.nan else x)
        df[f'{m}_y'] = df[f'{m}_y'].apply(lambda x: '' if x is np.nan else x)
        ## concatenating terms & cleaning columns
        try :
            df[f'{m}_x'] = df[f'{m}_x'] + df[f'{m}_y']    
        except:
            print(f"Error on column {m}")
        df.rename(columns = {f'{m}_x':f'{m}'}, inplace=True)
        del df[f'{m}_y']
    
    print(df_max_index)
    df['index'] = df['index'].astype('int')
    df['term'] = df['term'].astype('str')
    df.info()
    return df

#merging 2 dataframess at a time
def merge_dfs(df_x, df_y):
    print(f"merging based on '{merge_col}' column")
    df = pd.merge(
        df_x, #left source
        df_y, #right source
        how="outer",
        on=merge_col,
        left_on=None,
        right_on=None,
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=('_x', '_y'), #used to identify left and right sources in temp df
        copy=True,
        indicator=True
    )
    df.to_csv(f"{results_path}/{csv_out}_temp_with_duplicates.csv",index=False)
    df.drop_duplicates(subset=[merge_col], keep='first', inplace=True, ignore_index=True)
    df.info()
    df.to_csv(f"{results_path}/{csv_out}_temp_without_duplicates.csv",index=False)
    df = combine_columns(df)
    
    return df

In [ ]:
#merging
final_df=merge_dfs(df,df_add)

#cleaning years
final_df['year']=final_df['year'].apply(lambda y: 0 if np.isnan(y) else int(y))

print(final_df)

#droping the _merge column, not useful if everything went ok
#final_df.drop(columns='_merge', inplace=True, axis=1)

In [ ]:
#manual conformity with destination column order
final_df=final_df[['index','path','filename','title','year','doi link','term','foundText','_merge']]
final_df.info()

In [ ]:
#creating output file
final_df.to_csv(f"{results_path}/{csv_out}.csv",index=False)